In [2]:
# کتاب خانه های مورد نیاز را وارد می کنیم
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm
import re

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
def remove_emoji(inputString):
  emoji_pattern = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    u"\U0001f926-\U0001f937"
    u'\U00010000-\U0010ffff'
    u"\u200d"
    u"\u2640-\u2642"
    u"\u2600-\u2B55"
    u"\u23cf"
    u"\u23e9"
    u"\u231a"
    u"\u3030"
    u"\ufe0f"
    u"\u2069"
    u"\u2066"
    u"\u200c"
    u"\u2068"
    u"\u2067"
    "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', inputString)


def normaliz_persian_comments(my_text):
  # حروف انگلیسی را حذف می کنیم
  # my_text = my_text.lower().replace("a" , "").replace("b" , "").replace("c" , "").replace("d" , "").replace("e" , "").replace("f" , "").replace("g" , "").replace("h" , "").replace("i" , "").replace("j" , "").replace("k" , "").replace("l" , "").replace("m" , "").replace("n" , "").replace("o" , "").replace("p" , "").replace("q" , "").replace("r" , "").replace("s" , "").replace("t" , "").replace("u" , "").replace("v" , "").replace("w" , "").replace("x" , "").replace("y" , "").replace("z" , "")

  # اعداد فارسی و انگلیسی حذف می کنیم
  my_text = my_text.replace("۱","").replace("۲","").replace("۳","").replace("۴","").replace("۵","").replace("۶","").replace("۷","").replace("۸","").replace("۹","").replace("۰","").replace("1","").replace("2","").replace("3","").replace("4","").replace("5","").replace("6","").replace("7","").replace("8","").replace("9","").replace("0","")

  # نماد هارا حذف می کنیم
  my_text = my_text.replace("!","").replace(":" , "").replace("@","").replace("#","").replace("$","").replace("%","").replace("^","").replace("&","").replace("*","").replace("(","").replace(")","").replace("-","").replace("_","").replace("=","").replace("+","").replace("{","").replace("}","").replace("[","").replace("]","").replace("/","").replace("\\","").replace("  "," ").replace("ــ","ـ").replace("\n","").replace("\t","").replace("<","").replace(">","").replace(",","").replace("~","").replace("`","").replace("\"" , "").replace("'" , "")

  # حروف عربی را با فارسی جایگزین می کنیم و جلو تکرار حروف را می گیریم مثلا عاالیی را تبدیل به عالی می کنیم حروف صدادار می تواند چندین بار تکرار شود
  my_text = my_text.replace("ي","ی").replace("ك","ک").replace("،" , "").replace("اا","ا").replace("یی","ی").replace("اا","ا").replace("یی","ی").replace("." , "")
  # حذف نیم فاصله
  my_text = re.sub(r"\u200c" , " " , my_text)
  my_text = my_text.replace("‌" , " ")
  # حذف فاصله های اضافی
  my_text = re.sub(r"\s+" , " " , my_text).strip()
  # فقط حروف و فاصله را نگه می دارد
  # my_text = re.sub(r"[^\w\s]" , "" , my_text)
  # حذف اعداد
  my_text = re.sub(r"\d+" , "" , my_text)
  # برای اطمینان یک بار با کتاب خانه هزم نرمالیز می کنیم
  # if "ــ" in my_text: my_text = Normalizer().normalize(my_text)
  # حروف کشیده را کنترل می کنیم
  if "ــ" in my_text:
    while True:
      my_text = my_text.replace("ــ","")
      if "ــ" not in my_text: break
  return my_text

# اضافه کردن متنی در یک متن
def insert_str (source_str, insert_str, pos):
  return source_str[:pos] + insert_str + source_str[pos:]

def question_mark(text):
  temp = ""
  for i in text:
    if i == "?" or i == "؟" or i == "؟":
      temp += " "
      temp += "?"
      temp += " "
    else:
      temp += i
  return temp

def nogh_mark(text):
  temp = ""
  for i in text:
    if i == ".":
      temp += " "
      temp += "."
      temp += " "
    else:
      temp += i
  return temp

def hich(text):
  if len(text) == 0:
    return "هیچ"
  else:
    return text


In [5]:
# دیتاست را ایمپورت می کنیم
data = pd.read_csv("/content/drive/MyDrive/Tickets (1).csv")

In [6]:
data

,TicketId,Text
0,1,سلام
1,1,تست
2,2,تست
3,2,dfgdfg
4,3,تست
...,...,...
8050,3113,وقتتون بخیر ، پیامک همینه؟
8051,3113,خیلی ممنون
8052,3115,سلام وقت بخیر بهترین سن برای شروع یادگیری هوش ...
8053,3116,سلام وقتتون بخیر،من تابستون در حال تکمیل دوره ...


از آنجایی که دیتاست برچسب ندارد 150 تا از این دیتاست برمی داریم و به صورت دستی لیبل می زنیم و حال آن را ایمپورت می کنیم

In [7]:
df_label = pd.read_csv("/content/drive/MyDrive/data_label.csv")

In [8]:
df_label

,index,TicketId,Text,Essential,Not essential,Educational,Not educational
0,0,1,سلام,0,1,0,1
1,1,1,تست,0,1,0,1
2,2,2,تست,0,1,0,1
3,3,2,dfgdfg,0,1,0,1
4,4,3,تست,0,1,0,1
...,...,...,...,...,...,...,...
145,8050,3113,وقتتون بخیر ، پیامک همینه؟,1,0,0,1
146,8051,3113,خیلی ممنون,0,1,0,1
147,8052,3115,سلام وقت بخیر بهترین سن برای شروع یادگیری هوش ...,1,0,1,0
148,8053,3116,سلام وقتتون بخیر،من تابستون در حال تکمیل دوره ...,1,0,0,1


دسته بندی به این شکل است:
که آیا تیکت ضروری است یا نه تیکت های ضروری جمله های سوالی است و تیکت های غیر ضروری کلماتی مانند ممنون است که نیازی نیست اوپراتور به صورت فنی به آن جواب دهد و می توان از یک مدل ساده به جای آن استفاده کرد
دسته بندی دیگر تعیین می کند که آیا تیکت سوالی است یا خیر

In [9]:
# چک می کنیم که دیتاست داده ازدست رفته نداشته باشد
print(data.isna().sum())

TicketId    0
Text        0
dtype: int64


In [10]:
# و حالا مراحل پیش پردازش را شروع می کنیم

# حذف اموجی ها
df_label["Text"] = df_label["Text"].apply(remove_emoji)

# حذف اعداد علامات و فاصله های اضافی
df_label["Text"] = df_label["Text"].apply(normaliz_persian_comments)

df_label["Text"] = df_label["Text"].apply(question_mark)

df_label["Text"] = df_label["Text"].apply(nogh_mark)

df_label["Text"] = df_label["Text"].apply(hich)


# و حالا برای داده اصلی
data["Text"] = data["Text"].apply(remove_emoji)

# حذف اعداد علامات و فاصله های اضافی
data["Text"] = data["Text"].apply(normaliz_persian_comments)

data["Text"] = data["Text"].apply(question_mark)

data["Text"] = data["Text"].apply(nogh_mark)

data["Text"] = data["Text"].apply(hich)

از آنجایی که نقطه و علامت سوال می توانند در دسته بندی به ما کمک کنند آن هارا حذف نمی کنیم و نگه می داریم

In [11]:
big_data_comment = list(data["Text"])
data_comment = list(df_label["Text"])
data_zar = df_label["Essential"].values.reshape(-1,1)
# data_zar = list(df_label["Essential"])

In [12]:
# برای دقیق تر شدن توکن آیزر از یک دیتاست 20000 کلمه ای فارسی استفاده می کنیم
word = open("/content/drive/MyDrive/parsi_worder.txt" , "r" , encoding = "utf-8")
words = word.readlines()
word.close()

In [13]:
# از آنجایی که حروف کلمات این توکن آیزر دارای بک اسلش ان است از دستور زیر استفاده می کنیم تا بک اسلش هارا حذف کنیم
for i in range(len(words)):words[i] = words[i].strip()

In [14]:
# برای اینکه مطمعن شویم در لغت نامه توکن آیزر کلمه تکراری وجود ندارد اول از مجموعه استفاده می کنیم و در مرحله بعد مجموعه را تبدیل به لیست می کنیم
worder = set(words)
for i in big_data_comment:
    for i in i.split():
        worder.add(i)

In [15]:
worder = list(worder)
worder.insert(0 , "<pad>")
worder.insert(1 , "<unk>")
# worder.insert(2 , "?")

In [16]:
# با استفاده از لغت نامه یک دیکشنری می سازیم که هر کلمه را به ایندکسش متناظر کند
word2index = {a : b for b,a in enumerate(worder)}

In [17]:
len(word2index)

27825

In [18]:
# و حالا توکن آیزر خود را می سازیم
def Tokenizer(text):
    temp = list()
    for i in text.split():
        if i in word2index:
            temp.append(word2index[i])
        else:
            temp.append(word2index["<unk>"])
    return temp

In [19]:
# ورودی تابع زیر یک لیست از نظرات است که به عنوان خروجی یک لیست از توکن آیز شده جملات می دهد
def Token_list(text_l):
    temp = list()
    for i in text_l:
        temp.append(Tokenizer(i))
    return temp

In [20]:
index_data = Token_list(data_comment)

In [21]:
# با استفاده از تابع زیر توکن آیز شده جملات را پدینگ می کنیم
maxi_l = max([len(i) for i in big_data_comment])
def pading_data(index_Data):
    temp = list()
    tempy = list()
    for i in index_Data:
        for d in i:
            tempy.append(d)
        for i in range(maxi_l - len(tempy)):
            tempy.append(0)
        temp.append(tempy)
        tempy = list()
    return temp

In [22]:
index_data_pading = pading_data(index_data)

In [23]:
# و بردار توکن آیز شده را تبدیل به تنسور می کنیم
index_data_pading_tensor = torch.tensor(index_data_pading)#,dtype = torch.float32
index_label_pading_tensor = torch.tensor(data_zar,dtype = torch.float32)#

In [24]:
# شبکه خود را می سازیم
# برای این مسئله ما از شبکه آر ان ان استفاده می کنیم
class My_Rnn(nn.Module):
  def __init__(self , vocab_size , embedding_dim , hidden_dim , num_layers):
    super().__init__()
    # ورودی لایه ی امبدینگ را به اندازه طول لغت نامه تعیین می کنیم
    self.embedding = nn.Embedding(vocab_size , embedding_dim , padding_idx=0)
    # از شبکه آر ان ان استفاده می کنیم
    self.rnn = nn.RNN(embedding_dim , hidden_dim , num_layers , batch_first=True)
    # خروجی شبکه آر ان ان را به یک لایه ی لینیر می دهیم و به یک بردار سه تایی می بریم
    self.linear = nn.Linear(hidden_dim , 1)
    # و درنهایت خروجی لایه ی لینیر را به یک تابع فعال ساز سیگمویید می دهیم تا خروجی بین صفر و یک باشد
    self.sigmoid = nn.Sigmoid()

  def forward(self , x):
    x = self.embedding(x)
    _,h_n = self.rnn(x)
    end = self.linear(h_n[-1])
    end = self.sigmoid(end)
    return end

In [25]:
# پارامتر های مودل را تعیین می کنیم
vocab_size = len(word2index)
embedding_dim = 128
hidden_dim = 512
num_layers = 4
model = My_Rnn(vocab_size , embedding_dim , hidden_dim , num_layers)

In [26]:
# دیتالودر و اوپتی مایزر آدام را وارد می کنیم
from torch.utils.data import DataLoader , TensorDataset
from torch.optim import Adam

In [27]:
# دیتا لودر را می سازیم و بتچ سایز را برابر 32 قرار می دهیم
mybatch_size = 32
maki_data = TensorDataset(index_data_pading_tensor , index_label_pading_tensor)
dataloader = DataLoader(maki_data , batch_size = mybatch_size)

In [28]:
loss_fu = nn.BCELoss()
opt = Adam(model.parameters() , lr = 0.01)

# آموزش مدل برای دسته بندی اول

In [72]:
num_epoch = 6
for epoch in range(num_epoch):
  sum_loss = 0
  for i , batch in enumerate(dataloader):
    opt.zero_grad()
    inputs = batch[0]
    target = batch[1]

    y = model(inputs)

    loss_value = loss_fu(y , target)
    loss_value.backward()

    opt.step()

    sum_loss += loss_value.item()
  print(f"Epoch : {epoch + 1} , Loss : {sum_loss / len(dataloader)}")

Epoch : 1 , Loss : 1.5059223532676698
Epoch : 2 , Loss : 0.8621351420879364
Epoch : 3 , Loss : 0.927574634552002
Epoch : 4 , Loss : 0.6827068626880646
Epoch : 5 , Loss : 0.6060752809047699
Epoch : 6 , Loss : 0.6174580156803131


In [73]:
x,y = next(iter(dataloader))

In [88]:
y_hat = model(x)

In [89]:
(y_hat.round() == y)

tensor([[False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [False],
        [False],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [False],
        [ True],
        [ True],
        [False],
        [ True],
        [False],
        [False],
        [ True],
        [ True],
        [False],
        [ True]])

In [ ]:
model_scripted = torch.jit.script(model)
model_scripted.save('model_Essential.pt')

In [78]:
# model2 = torch.jit.load('model_scripted.pt')
# model2.eval()

# آموزش مدل برای دسته بندی دوم

In [29]:
label = df_label["Educational"].values.reshape(-1,1)

In [30]:
label_tensor = torch.tensor(label,dtype = torch.float32)

In [31]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score , classification_report

In [32]:
my_data = pd.DataFrame(index_data_pading_tensor)
my_targ = pd.DataFrame(label_tensor)

In [33]:
clf = SVC(kernel = "poly")
clf.fit(my_data,my_targ)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(kernel='poly')

In [35]:
predicted = clf.predict(my_data)
ac = accuracy_score(my_targ , predicted)
print("accuracy model2(entropy) = : {}%".format(ac * 100))

accuracy model2(entropy) = : 86.0%


# وحالا تابعی که فایل سی اس وی کامنت هارا می سازد را می نویسیم

In [56]:
def cl_ti(data):
  model2 = torch.jit.load('/content/drive/MyDrive/model_Essential.pt')
  model2.eval()
  copy_comment = list(data["Text"])
  # و حالا برای داده اصلی
  data["Text"] = data["Text"].apply(remove_emoji)

  # حذف اعداد علامات و فاصله های اضافی
  data["Text"] = data["Text"].apply(normaliz_persian_comments)

  data["Text"] = data["Text"].apply(question_mark)

  data["Text"] = data["Text"].apply(nogh_mark)

  data["Text"] = data["Text"].apply(hich)
  comment = list(data["Text"])
  token_list = pading_data(Token_list(comment))
  token_list = torch.tensor(token_list)
  y_hat = model2(token_list)

  y_hat2 = clf.predict(token_list)
  Essential = list(y_hat.round())
  Not_Essential = [0 if i == 1 else 1 for i in Essential]
  Educational = list(y_hat2)
  Not_Educational = [0 if i == 1 else 1 for i in Educational]

  temp = {
      "ticket_id" : [i+1 for i in range(len(copy_comment))],
      "text" : copy_comment,
      "Essential" : Essential,
      "Not_Essential" : Not_Essential,
      "Educational" : Educational,
      "Not_Educational" : Not_Educational
  }
  temp = pd.DataFrame(temp)
  return temp.set_index("ticket_id")


In [41]:
data = pd.read_csv("/content/drive/MyDrive/Tickets (1).csv")

In [42]:
data

,TicketId,Text
0,1,سلام
1,1,تست
2,2,تست
3,2,dfgdfg
4,3,تست
...,...,...
8050,3113,وقتتون بخیر ، پیامک همینه؟
8051,3113,خیلی ممنون
8052,3115,سلام وقت بخیر بهترین سن برای شروع یادگیری هوش ...
8053,3116,سلام وقتتون بخیر،من تابستون در حال تکمیل دوره ...


In [43]:
copy_comment = list(data["Text"])

In [44]:
# و حالا برای داده اصلی
data["Text"] = data["Text"].apply(remove_emoji)

# حذف اعداد علامات و فاصله های اضافی
data["Text"] = data["Text"].apply(normaliz_persian_comments)

data["Text"] = data["Text"].apply(question_mark)

data["Text"] = data["Text"].apply(nogh_mark)

data["Text"] = data["Text"].apply(hich)

In [45]:
model2 = torch.jit.load('/content/drive/MyDrive/model_Essential.pt')
model2.eval()

RecursiveScriptModule(
  original_name=My_Rnn
  (embedding): RecursiveScriptModule(original_name=Embedding)
  (rnn): RecursiveScriptModule(original_name=RNN)
  (linear): RecursiveScriptModule(original_name=Linear)
  (sigmoid): RecursiveScriptModule(original_name=Sigmoid)
)

In [ ]:
cv = pd.read_csv("Tickets - Phase2")

In [82]:
comment = list(cv["Text"])
token_list = pading_data(Token_list(comment))
token_list = torch.tensor(token_list)


In [ ]:
y_hat = model2(token_list)

In [60]:
y_hat2 = clf.predict(token_list)

In [76]:
list(y_hat2)

[0.0, 0.0, 0.0, 0.0, 0.0]

In [78]:
a1 = [i.item() for i in list(y_hat.round())]
temp = {
      "ticket_id" : [i for i in range(len(comment))],
      "text" : comment,
      "Essential" : a1,
      "Not_Essential" : [1 if i == 0 else 0 for i in a1],
      "Educational" : list(y_hat2),
      "Not_Educational" : [1 if i == 0 else 0 for i in list(y_hat2)],
  }


In [79]:
temp = pd.DataFrame(temp)